In [1]:
import pandas as pd
import numpy as np
from scipy import stats
import ast
import networkx as nx
from pyvis.network import Network
import graphviz
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use('tableau-colorblind10')
import numba
import seaborn as sns
import iqplot
from sklearn import metrics
import bokeh.models
import bokeh.plotting
import bokeh.io
bokeh.io.output_notebook()

Loading BokehJS ...

In [2]:
ec1 = '#1A85FF'
ec0 = '#D41159'

In [3]:
beta25 = pd.read_csv('data/SERGIO/RunConsistancySERGIO25beta.csv')
beta25 = beta25[beta25['Run'] == 28]
beta25['F1'] = (2 * beta25['Torch Precision'] * beta25['Torch Recall']) / (beta25['Torch Precision'] + beta25['Torch Recall'])
print(list(beta25['F1']))
print(list(beta25['Torch Accuracy']))
gnn25 = pd.read_csv('data/SERGIO/RunConsistancySERGIO25gnn.csv')
gnn25 = gnn25[gnn25['Run'] == 1]
gnn25['F1'] = (2 * gnn25['Torch Precision'] * gnn25['Torch Recall']) / (gnn25['Torch Precision'] + gnn25['Torch Recall'])
print(list(gnn25['F1']))
print(list(gnn25['Torch Accuracy']))

[0.953846153127574]
[0.980000019]
[0.11764705882352941]
[0.800000012]


In [4]:
ungnn25 = pd.read_csv('data/SERGIO/RunProbsVSGTSERGIO25gnn.csv')
grgnn25 = ungnn25[ungnn25['Run'] == 1]
ungnn50 = pd.read_csv('data/SERGIO/SERGIO50GNNRunProbsVSGTCheck.csv')
grgnn50 = ungnn50[ungnn50['Run'] == 40]

In [5]:
unbeta25 = pd.read_csv('data/SERGIO/RunProbsVSGTSERGIO25beta.csv')
grbeta25 = unbeta25[unbeta25['Run'] == 28]
unbeta50 = pd.read_csv('data/SERGIO/RunProbsVSGTSERGIO50beta.csv')
grbeta50 = unbeta50[unbeta50['Run'] == 11]

In [6]:
unbeta25['Category'] = unbeta25['GT'].apply(lambda x: 'False Positive' if x == 0 else 'True Positive')
unbeta50['Category'] = unbeta50['GT'].apply(lambda x: 'False Positive' if x == 0 else 'True Positive')

In [7]:
tp = unbeta25[unbeta25['Category'] == 'True Positive']
t = list(tp['Probability'])
fp = unbeta25[unbeta25['Category'] == 'False Positive']
f = list(fp['Probability'])
stat, mu = stats.mannwhitneyu(t, f)
stat, k = stats.kruskal(t, f)
print(mu, k)

0.0 0.0


In [8]:
tp = unbeta50[unbeta50['Category'] == 'True Positive']
t = list(tp['Probability'])
fp = unbeta50[unbeta50['Category'] == 'False Positive']
f = list(fp['Probability'])
stat, mu = stats.mannwhitneyu(t, f)
stat, k = stats.kruskal(t, f)
print(mu, k)

0.0 0.0


In [9]:
tp_edges = grgnn25[grgnn25['GT'] == 1]
tp_set = set()
p1s = list(tp_edges['P1'])
p2s = list(tp_edges['P2'])
for i in range(len(p1s)):
    p1 = p1s[i]
    p2 = p2s[i]
    tp_set.add((p1, p2))
g = Network(notebook=True, directed=True)
ei1 = list(grgnn25['P1'])
ei2 = list(grgnn25['P2'])
prob = list(grgnn25['Probability'])
b1 = []
b2 = []
n = []
c = []
num = 0
denom = 0
for i in range(len(ei1)):
    denom += 1
    if prob[i] >= 0.5:
        num += 1
        b1.append(ei1[i])
        b2.append(ei2[i])
u1 = np.unique(b1)
u2 = np.unique(b2)
for i in u1:
    if i not in n:
        n.append(i)
        c.append('black')
for i in u2:
    if i not in n:
        n.append(i)
        c.append('black')
g.add_nodes(n, color=c)
for i in range(0, len(b1)):
    g.add_edge(b1[i], b2[i], width=5)
for edge in g.get_edges():
    p1 = edge['from']
    p2 = edge['to']
    if (p1, p2) in tp_set:
        edge["color"] = ec1
    else:
        edge["color"] = ec0
g.show('data/Vis/SGNN25.html')

data/Vis/SGNN25.html


In [10]:
print(num / denom)

0.013333333333333334


In [11]:
tp_edges = grbeta25[grbeta25['GT'] == 1]
tp_set = set()
p1s = list(tp_edges['P1'])
p2s = list(tp_edges['P2'])
for i in range(len(p1s)):
    p1 = p1s[i]
    p2 = p2s[i]
    tp_set.add((p1, p2))
g = Network(notebook=True, directed=True)
ei1 = list(grbeta25['P1'])
ei2 = list(grbeta25['P2'])
prob = list(grbeta25['Probability'])
b1 = []
b2 = []
n = []
c = []
num = 0
denom = 0
width = dict()
wtr = set()
for i in range(len(ei1)):
    denom += 1
    if prob[i] >= 0.5:
        num += 1
        b1.append(ei1[i])
        b2.append(ei2[i])
        if (ei1[i], ei2[i]) not in wtr:
            if ei1[i] in width.keys():
                width[ei1[i]][ei2[i]] = prob[i]
            else:
                width[ei1[i]] = {ei2[i]: prob[i]}
            wtr.add((ei1[i], ei2[i]))
u1 = np.unique(b1)
u2 = np.unique(b2)
for i in u1:
    if i not in n:
        n.append(i)
        c.append('black')
for i in u2:
    if i not in n:
        n.append(i)
        c.append('black')
g.add_nodes(n, color=c)
for i in range(0, len(b1)):
    w = 5
    g.add_edge(b1[i], b2[i], width=w)
for edge in g.get_edges():
    p1 = edge['from']
    p2 = edge['to']
    if (p1, p2) in tp_set:
        edge["color"] = ec1
    else:
        edge["color"] = ec0
g.show('data/Vis/SBeta25.html')

data/Vis/SBeta25.html


In [12]:
tp_edges = grbeta25[grbeta25['GT'] == 1]
tp_set = set()
p1s = list(tp_edges['P1'])
p2s = list(tp_edges['P2'])
for i in range(len(p1s)):
    p1 = p1s[i]
    p2 = p2s[i]
    tp_set.add((p1, p2))
g = Network(notebook=True, directed=True)
ei1 = list(grbeta25['P1'])
ei2 = list(grbeta25['P2'])
prob = list(grbeta25['Probability'])
b1 = []
b2 = []
n = []
c = []
mx=0
mn=1
width = dict()
wtr = set()
for i in range(len(ei1)):
    if prob[i] >= 0.5:
        if mx < prob[i]:
            mx = prob[i]
        if mn > prob[i]:
            mn = prob[i]
        b1.append(ei1[i])
        b2.append(ei2[i])
        if (ei1[i], ei2[i]) not in wtr:
            if ei1[i] in width.keys():
                width[ei1[i]][ei2[i]] = prob[i]
            else:
                width[ei1[i]] = {ei2[i]: prob[i]}
            wtr.add((ei1[i], ei2[i]))
u1 = np.unique(b1)
u2 = np.unique(b2)
for i in u1:
    if i not in n:
        n.append(i)
        c.append('black')
for i in u2:
    if i not in n:
        n.append(i)
        c.append('black')
g.add_nodes(n, color=c)
for i in range(0, len(b1)):
    w = 5 * (width[b1[i]][b2[i]] - mn + 1e-5) / (mx - mn)
    if w <= 0:
        print(w)
    g.add_edge(b1[i], b2[i], width=w)
for edge in g.get_edges():
    p1 = edge['from']
    p2 = edge['to']
    edge["color"] = 'black'
g.show('data/Vis/SBeta25Weight.html')

data/Vis/SBeta25Weight.html


In [13]:
tp_edges = grbeta25[grbeta25['GT'] == 1]
tp_set = set()
p1s = list(tp_edges['P1'])
p2s = list(tp_edges['P2'])
for i in range(len(p1s)):
    p1 = p1s[i]
    p2 = p2s[i]
    tp_set.add((p1, p2))
g = Network(notebook=True, directed=True)
ei1 = list(grbeta25['P1'])
ei2 = list(grbeta25['P2'])
prob = list(grbeta25['Probability'])
b1 = []
b2 = []
n = []
c = []
lst = []
width = dict()
wtr = set()
for i in range(len(ei1)):
    if prob[i] >= 0.5:
        lst.append(prob[i])
        b1.append(ei1[i])
        b2.append(ei2[i])
        if (ei1[i], ei2[i]) not in wtr:
            if ei1[i] in width.keys():
                width[ei1[i]][ei2[i]] = prob[i]
            else:
                width[ei1[i]] = {ei2[i]: prob[i]}
            wtr.add((ei1[i], ei2[i]))
u1 = np.unique(b1)
u2 = np.unique(b2)
for i in u1:
    if i not in n:
        n.append(i)
        c.append('black')
for i in u2:
    if i not in n:
        n.append(i)
        c.append('black')
g.add_nodes(n, color=c)
for i in range(0, len(b1)):
    w = 5 * (width[b1[i]][b2[i]] - np.mean(lst)) / np.std(lst)
    g.add_edge(b1[i], b2[i], width=w)
for edge in g.get_edges():
    p1 = edge['from']
    p2 = edge['to']
    edge["color"] = 'black'
g.show('data/Vis/SBeta25Weight.html')

data/Vis/SBeta25Weight.html


In [14]:
tp_edges = grbeta50[grbeta50['GT'] == 1]
tp_set = set()
p1s = list(tp_edges['P1'])
p2s = list(tp_edges['P2'])
for i in range(len(p1s)):
    p1 = p1s[i]
    p2 = p2s[i]
    tp_set.add((p1, p2))
g = Network(notebook=True, directed=True)
ei1 = list(grbeta25['P1'])
ei2 = list(grbeta25['P2'])
prob = list(grbeta25['Probability'])
b1 = []
b2 = []
n = []
c = []
mx=0
mn=1
width = dict()
wtr = set()
for i in range(len(ei1)):
    if prob[i] >= 0.5:
        if prob[i] > mx:
            mx = prob[i]
        if mn > prob[i]:
            mn = prob[i]
        b1.append(ei1[i])
        b2.append(ei2[i])
        if (ei1[i], ei2[i]) not in wtr:
            if ei1[i] in width.keys():
                width[ei1[i]][ei2[i]] = prob[i]
            else:
                width[ei1[i]] = {ei2[i]: prob[i]}
            wtr.add((ei1[i], ei2[i]))
u1 = np.unique(b1)
u2 = np.unique(b2)
for i in u1:
    if i not in n:
        n.append(i)
        c.append('black')
for i in u2:
    if i not in n:
        n.append(i)
        c.append('black')
g.add_nodes(n, color=c)
for i in range(0, len(b1)):
    w = 5 * (width[b1[i]][b2[i]] - mn + 1e-5) / (mx - mn)
    if w <= 0:
        print(w)
    g.add_edge(b1[i], b2[i], width=w)
for edge in g.get_edges():
    p1 = edge['from']
    p2 = edge['to']
    edge["color"] = 'black'
g.show('data/Vis/SBeta50Weight.html')

data/Vis/SBeta50Weight.html


In [15]:
tp_edges = grbeta25[grbeta25['GT'] == 1]
tp_set = set()
p1s = list(tp_edges['P1'])
p2s = list(tp_edges['P2'])
for i in range(len(p1s)):
    p1 = p1s[i]
    p2 = p2s[i]
    tp_set.add((p1, p2))
g = Network(notebook=True, directed=True)
ei1 = list(grbeta25['P1'])
ei2 = list(grbeta25['P2'])
prob = list(grbeta25['Probability'])
b1 = []
b2 = []
n = []
c = []
lst = []
width = dict()
wtr = set()
for i in range(len(ei1)):
    if prob[i] >= 0.5:
        lst.append(prob[i])
        b1.append(ei1[i])
        b2.append(ei2[i])
        if (ei1[i], ei2[i]) not in wtr:
            if ei1[i] in width.keys():
                width[ei1[i]][ei2[i]] = prob[i]
            else:
                width[ei1[i]] = {ei2[i]: prob[i]}
            wtr.add((ei1[i], ei2[i]))
u1 = np.unique(b1)
u2 = np.unique(b2)
for i in u1:
    if i not in n:
        n.append(i)
        c.append('black')
for i in u2:
    if i not in n:
        n.append(i)
        c.append('black')
g.add_nodes(n, color=c)
for i in range(0, len(b1)):
    w = 5 * (width[b1[i]][b2[i]] - np.mean(lst)) / np.std(lst)
    g.add_edge(b1[i], b2[i], width=w)
for edge in g.get_edges():
    p1 = edge['from']
    p2 = edge['to']
    if (p1, p2) in tp_set:
        edge["color"] = lighten(ec1, width[p1][p2])
    else:
        edge["color"] = lighten(ec0, width[p1][p2])
g.show('data/Vis/SBeta25WeightShade.html')

NameError: name 'lighten' is not defined

In [ ]:
tp_edges = grbeta50[grbeta50['GT'] == 1]
tp_set = set()
p1s = list(tp_edges['P1'])
p2s = list(tp_edges['P2'])
for i in range(len(p1s)):
    p1 = p1s[i]
    p2 = p2s[i]
    tp_set.add((p1, p2))
g = Network(notebook=True, directed=True)
ei1 = list(grbeta50['P1'])
ei2 = list(grbeta50['P2'])
prob = list(grbeta50['Probability'])
b1 = []
b2 = []
n = []
c = []
num = 0
denom = 0
width = dict()
wtr = set()
for i in range(len(ei1)):
    denom += 1
    if prob[i] >= 0.5:
        num += 1
        b1.append(ei1[i])
        b2.append(ei2[i])
        if (ei1[i], ei2[i]) not in wtr:
            if ei1[i] in width.keys():
                width[ei1[i]][ei2[i]] = prob[i]
            else:
                width[ei1[i]] = {ei2[i]: prob[i]}
            wtr.add((ei1[i], ei2[i]))
u1 = np.unique(b1)
u2 = np.unique(b2)
for i in u1:
    if i not in n:
        n.append(i)
        c.append('black')
for i in u2:
    if i not in n:
        n.append(i)
        c.append('black')
g.add_nodes(n, color=c)
for i in range(0, len(b1)):
    w = 5 * width[b1[i]][b2[i]]
    g.add_edge(b1[i], b2[i], width=w)
for edge in g.get_edges():
    p1 = edge['from']
    p2 = edge['to']
    edge["color"] = lighten('#000000', width[p1][p2])
g.show('data/Vis/SBeta50Weight.html')

In [ ]:
print(num / denom)

In [ ]:
g = Network(notebook=True, directed=True)
ei1 = list(grgnn25['P1'])
ei2 = list(grgnn25['P2'])
prob = list(grgnn25['GT'])
b1 = []
b2 = []
n = []
c = []
for i in range(len(ei1)):
    if prob[i] == 1:
        b1.append(ei1[i])
        b2.append(ei2[i])
u1 = np.unique(b1)
u2 = np.unique(b2)
for i in u1:
    if i not in n:
        n.append(i)
        c.append('black')
for i in u2:
    if i not in n:
        n.append(i)
        c.append('black')
g.add_nodes(n, color=c)
for i in range(0, len(b1)):
    g.add_edge(b1[i], b2[i], width=5)
for edge in g.get_edges():
    edge["color"] = ec1
g.show('data/Vis/SGT25.html')

In [ ]:
tp_edges = grgnn50[grgnn50['GT'] == 1]
tp_set = set()
p1s = list(tp_edges['P1'])
p2s = list(tp_edges['P2'])
for i in range(len(p1s)):
    p1 = p1s[i]
    p2 = p2s[i]
    tp_set.add((p1, p2))
g = Network(notebook=True, directed=True)
ei1 = list(grgnn50['P1'])
ei2 = list(grgnn50['P2'])
prob = list(grgnn50['Probability'])
b1 = []
b2 = []
n = []
c = []
num = 0
denom = 0
for i in range(len(ei1)):
    denom += 1
    if prob[i] >= 0.5:
        num += 1
        b1.append(ei1[i])
        b2.append(ei2[i])
u1 = np.unique(b1)
u2 = np.unique(b2)
for i in u1:
    if i not in n:
        n.append(i)
        c.append('black')
for i in u2:
    if i not in n:
        n.append(i)
        c.append('black')
g.add_nodes(n, color=c)
for i in range(0, len(b1)):
    g.add_edge(b1[i], b2[i], width=5)
for edge in g.get_edges():
    p1 = edge['from']
    p2 = edge['to']
    if (p1, p2) in tp_set:
        edge["color"] = ec1
    else:
        edge["color"] = ec0
g.show('data/Vis/SGNN50.html')

In [ ]:
print(num / denom)

In [ ]:
tp_edges = grbeta50[grbeta50['GT'] == 1]
tp_set = set()
p1s = list(tp_edges['P1'])
p2s = list(tp_edges['P2'])
for i in range(len(p1s)):
    p1 = p1s[i]
    p2 = p2s[i]
    tp_set.add((p1, p2))
g = Network(notebook=True, directed=True)
ei1 = list(grbeta50['P1'])
ei2 = list(grbeta50['P2'])
prob = list(grbeta50['Probability'])
b1 = []
b2 = []
n = []
c = []
num = 0
denom = 0
width = dict()
wtr = set()
for i in range(len(ei1)):
    denom += 1
    if prob[i] >= 0.5:
        num += 1
        b1.append(ei1[i])
        b2.append(ei2[i])
        if (ei1[i], ei2[i]) not in wtr:
            if ei1[i] in width.keys():
                width[ei1[i]][ei2[i]] = prob[i]
            else:
                width[ei1[i]] = {ei2[i]: prob[i]}
            wtr.add((ei1[i], ei2[i]))
u1 = np.unique(b1)
u2 = np.unique(b2)
for i in u1:
    if i not in n:
        n.append(i)
        c.append('black')
for i in u2:
    if i not in n:
        n.append(i)
        c.append('black')
g.add_nodes(n, color=c)
for i in range(0, len(b1)):
    w = 5
    g.add_edge(b1[i], b2[i], width=w)
for edge in g.get_edges():
    p1 = edge['from']
    p2 = edge['to']
    if (p1, p2) in tp_set:
        edge["color"] = ec1
    else:
        edge["color"] = ec0
g.show('data/Vis/SBeta50.html')

In [ ]:
print(num / denom)

In [ ]:
print(denom)

In [ ]:
g = Network(notebook=True, directed=True)
ei1 = list(grgnn50['P1'])
ei2 = list(grgnn50['P2'])
prob = list(grgnn50['GT'])
b1 = []
b2 = []
n = []
c = []
for i in range(len(ei1)):
    if prob[i] == 1:
        b1.append(ei1[i])
        b2.append(ei2[i])
u1 = np.unique(b1)
u2 = np.unique(b2)
for i in u1:
    if i not in n:
        n.append(i)
        c.append('black')
for i in u2:
    if i not in n:
        n.append(i)
        c.append('black')
g.add_nodes(n, color=c)
for i in range(0, len(b1)):
    g.add_edge(b1[i], b2[i], width=5)
for edge in g.get_edges():
    edge["color"] = ec1
g.show('data/Vis/SGT50.html')

In [ ]:
print(num / denom)

In [ ]:
p = iqplot.striphistogram(
    data=unbeta25,
    q="Probability",
    cats='Category',
    palette=['grey', 'green'],
)
bokeh.io.show(p)

In [ ]:
p = sns.ecdfplot(data=unbeta25, x="Probability", hue="Category", 
                 palette=['grey', 'green']).set_title('SERGIO 25% Sparsity eCDF')

In [ ]:
p = iqplot.striphistogram(
    data=unbeta50,
    q="Probability",
    cats='Category',
    palette=['grey', 'green'],
)
bokeh.io.show(p)

In [ ]:
p = sns.ecdfplot(data=unbeta50, x="Probability", hue="Category", 
                 palette=['grey', 'green']).set_title('SERGIO 50% Sparsity eCDF')